# Week 1 - Retreiving and Preparing Text for Machines

This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze. This notebook outlines scraping text from the
web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Finally, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis.

For this notebook we will be using the following packages:

We made a python package just for this course: lucem_illud. If you haven't installed this package, you should run the following code first. You don't need to install the package later; all you need to do is just to import the package with: import lucem_illud. 

In [ ]:
!pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git
#installing lucem_illud package

If you're not familiar with jupyter notebook, you may wonder what the exclamation mark(!) at the beginning of the command does (or even what pip means). The exclamation mark enables us to execute Terminal commands in the notebook cells. 

In [6]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [26]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [594]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

ConnectionError: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /wiki/Content_analysis (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f89cfec1970>: Failed to establish a new connection: [Errno 65] No route to host'))

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [28]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Content analysis - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"X-4-xgpAAD8AAF2WHXgAAAAV","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":991020438,"wgRevisionId":991020438,"wgArticleId":473317,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles needing expert attention with no reason or talk parameter","Articles needing expert attention from April 2008","All articles needing expert attention","Sociology ar

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

But before we proceed to Beautiful Soup, a digression about Python syntax, especially about objects and functions.
For those who are not familiar with the syntax of python (or, if you're familiar with R programming), you might wonder what requests.get or wikiContentRequest.text mean. To understand this, you need to first understand what **objects** are. You may have heard that Python is an object oriented programming language (unlike the procedure oriented programming language, an example of which is R). Object is a set of variables (or, data) and functions into which you pass your data. So, in object oriented programming languages, like python, variables and functions are bunleded into objects.

For example, let's look at wikiContentRequest. We use dir() function, which returns the list of attributes and functions of objects.

In [8]:
 dir(wikiContentRequest)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

There's 'text' here. We used 'wikiContentRequest.text' to access 'text.' In other words, we use .(dot notation) to access functions from objects. wikiContentRequest has a set of functions, as shown above, and we used 'wikiContentRequest.text' to access one of them. By the way, dot notations do not necessarily refer to functions--it refers to anything that the entity contains. 



Moving on to the next step: BeautifulSoup, a Python library which extracts data from HTML and XML, and transforms HTML files into Python objects.

In [9]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia































Content analysis

From Wikipedia, the free encyclopedia



Jump to navigation
Jump to search
This article needs attention from an expert


This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

**Q: What is content_analysis_save? How does .html function?**

In [39]:
#content_analysis_save = 'wikipedia_content_analysis.html'
with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

open() is a function which literally opens and returns the file. This function has multiple modes, and, here, we used mode as 'w', which means: open a file for writing. And then, we use 'write' function to write on the empty file (content_analysis_save) that we created using open(content_analysis_save, mode='w', encoding='utf-8').} What did we write on this file? The text we got from wikiContentRequest.text

Now let's open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag. 

In [299]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

Content analysis is the study of documents and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.[1] One of the key advantages of using content analysis to analyse social phenomena is its non-invasive nature, in contrast to simulating social experiences or collecting survey answers.

Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of texts or artifacts which are assigned labels (sometimes called codes) to indicate the presence of interesting, meaningful pieces of content.[2][3] By systematically labeling the content of a set of texts, researchers can analyse patterns of content quantitatively using statistical methods, or use qualitative methods to analyse meanings of content within texts.

Computers are increasingly used in content analysis to auto

In [288]:
# list of <p> 中的部分
contentPTags 

[<div class="o-campaign-description"><div class="o-campaign-story mt3x" style="max-height:240px">Hello well-wishers. This is our donation page for 4-month-old baby Teera Kamat, born on August 14, 2020, diagnosed with Spinal Muscular Atrophy Type 1 (SMA1). All she asks is one gene; can you help us get it for her?<br/><br/>I am Rama, Teera’s aunt, who is helping my cousin Mihir and sister in law Priyanka raise the funds required to save our little angel's life. They have very limited financial support in Mumbai, India, where Teera is now located and hospitals are ill equipped to treat such genetic conditions. My family are fundraising in India through their local crowdfunding platforms, but they have tried very hard but have only raised 15% of their total goal. They really need our help, and the clock is running out for Teera as each day passes.<br/><br/>SMA is a terrible disease, a rarest of rare genetic disorder that affects 1 in 10000 kids worldwide. Teera is missing one gene in her b

Another excursion for those who are not familiar with programming: for loop. For loop is used to iterate over a sequence. "ContentPTags" contains multiple paragraphs, each of which starts and ends with `<p>`. What the "for pTag in contentPtags[:3]" does here is: find each paragraph in contentPTags, which, here, we limited to the first three using contentPtags[:3], and then print each paragraph. So, we have three paragraphs. By the way, you can insert `<p>` in juputer notebook!

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [317]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0   Content analysis is the study of documents and...
1   Practices and philosophies of content analysis...
2   Computers are increasingly used in content ana...
3   Content analysis is best understood as a broad...
4   The simplest and most objective form of conten...
5   A further step in analysis is the distinction ...
6   Quantitative content analysis highlights frequ...
7   Siegfried Kracauer provides a critique of quan...
8   More generally, content analysis is research u...
9   By having contents of communication available ...
10  Computer-assisted analysis can help with large...
11  Robert Weber notes: "To make valid inferences ...
12  There are five types of texts in content analy...
13  Over the years, content analysis has been appl...
14  In recent times, particularly with the advent ...
15  Quantitative content analysis has enjoyed a re...
16  Content analysis can also be described as stud...
17  Manifest content is read

Since we learned how to do for loop, you might get what we did here: using contentParagraphs = [], we made an empty list; and then, for each paragraph in contentPTags, we substituted every [\d+\] with '', i.e., removed every [\d+\], and then appended each paragraph (now without [\d+\]) to the empty list. As we can see, we have a dataframe, each row of which is each paragraph of contentPTags, without reference indicators. 

By the way, what does [\d+\] mean? If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

Now we have a `DataFrame` containing all relevant text from the page ready to be processed

In [318]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<re.Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [17]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [18]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

In [692]:
import pandas as pd

**HW1 Description:**  
The interest: how do the emotional descriptions and themes of medical crowdfunding (MCF) influence the donation outcome (number of donations, average donation amount)?   
Potentially, I can also see the distribution among Covid-related and non Covid-related campaigns.

Section1   
Collect the basic information of each campaign on the medical fundraiser page, such as title, amount raised, goal amount, and url, etc.  

In [7]:
gofundme_medical = "https://www.gofundme.com/discover/medical-fundraiser"
gofundRequest = requests.get(gofundme_medical)
gofundSoup = bs4.BeautifulSoup(gofundRequest.text, 'html.parser')
print(gofundSoup.text[:1000])

KeyboardInterrupt: 

In [695]:
# Campaign information is wrapped in <div> <class = "react-campaign-tile-details">
posts = []
medical_divTags = gofundSoup.body.findAll('div', {"class":"react-campaign-tile-details"})
for divTag in medical_divTags[:100]:
    posts.append(re.sub(r'\n','',divTag.text))  # delete all \n
print(posts)

### Variables of interest
# medical_campaign: the text of each campaign
posts_df = pd.DataFrame({'medical_campaign':posts})
# amount_raised: amount of money raised for now; amount_goal: amount of money for fundraising goal
posts_df['amount'] = posts_df.medical_campaign.apply(lambda x: re.findall(r'\$(.*) raised of (.*)\$', x))
posts_df['amount_raised'] = posts_df.amount.apply(lambda x: x[0][0])
posts_df['amount_goal'] = posts_df.amount.apply(lambda x: x[0][1].strip().replace("$",""))
# how long ago did the last donation conduct 
posts_df['last_donation'] = [i[0] for i in posts_df.medical_campaign.apply(lambda x: re.findall(r'Last donation (.*) ago', x))]
# whether the title and short description is covid related, roughly measured by whether it contains "Covid"
posts_df['covid_related'] = posts_df.medical_campaign.apply(lambda x: re.search(r'[Cc]vid|COVID', x) !=None)

['Kenosha, WI                            Justice for Jacob Blake                        Justice for Jacob Blake                    On August 23rd my son was shot multiple times in the back by a Kenosha Police Department officer, after my son broke up an altercation by an unrelated party. The shooting has now left my son critically injured as he fights for his life. The extent of my son’s injuries is unknown, but we remain pr...                                                            Last donation 4h ago                $2.3M raised of $3.0M                                            $2,342,199 raised                    ', 'New Milford, NJ                            PLEASE HELP ME IN THE BATTLE FOR MY LIFE                        PLEASE HELP ME IN THE BATTLE FOR MY LIFE                    PLEASE HELP ME IN THE BATTLE FOR MY LIFE.\xa0  My name is Mimosa Selimaj. I am a wife and proud mother of a beautiful 2-year-old boy, Dean, and we live in New Milford, a small town in northern New Jer

In [696]:
posts_df

,medical_campaign,amount,amount_raised,amount_goal,last_donation,covid_related
0,"Kenosha, WI Justice...","[(2.3M, $3.0M ...",2.3M,3.0M,4h,False
1,"New Milford, NJ PLE...","[(514,718, $600,000 ...","514,718","600,000",2h,False
2,"Sammamish, WA Can y...","[(464,153, $2.3M ...","464,153",2.3M,3m,False
3,"Eugene, OR Helping ...","[(423,560, $200,000 ...","423,560","200,000",2d,False
4,"Houston, TX Help Ma...","[(403,058, $1.0M ...","403,058",1.0M,3mos,False
5,"Rancho Santa Margarita, CA ...","[(376,271, $500,000 ...","376,271","500,000",4h,False
6,"Swedesboro, NJ Bria...","[(324,782, $1.5M ...","324,782",1.5M,4h,False
7,"El Paso, TX Save DO...","[(2.4M, $2.4M ...",2.4M,2.4M,1mo,False
8,"Pepsi-Cola Sign, NY ...","[(292,796, $250,000 ...","292,796","250,000",1d,True
9,"Westbury, NY Keith ...","[(289,172, $300,000 ...","289,172","300,000",2h,False


## Complement


# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [469]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Document', 0, 'documents'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Coding_(social_sciences)', 1, 'assigned labels (sometimes called codes)'), ('https://en.wikipedia.org/wiki/Semantics', 1, 'meaningful'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Quantitative_research', 1, 'quantitatively'), ('https://en.wikipedia.org/wiki/Statistics', 1, 'statistical methods'), ('https://en.wikipedia.org/wiki/Qualitative_research', 1, 'qualitative'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Machine_learning', 2, 'Machine learning')]


Another excursion: Why do we use enumerate() here? enumerate() takes a collection, enumerates, and returns an enumate object with both the numbers and the collection. For example, contentPTags (the collection we used here) is comprised of paragraphs. We want the paragraph number of each paragraph. And this is what enumerate() does: it returns the paragraph number and the paragraph. 

We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [470]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text']) # all wiki/content_analysis
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text'])) # 0 ~ len

contentParagraphsDF

,paragraph-text,source-paragraph-number,source-paragraph-text,source,paragraph-number
0,Content analysis is the study of documents and...,None,None,https://en.wikipedia.org/wiki/Content_analysis,0
1,Practices and philosophies of content analysis...,None,None,https://en.wikipedia.org/wiki/Content_analysis,1
2,Computers are increasingly used in content ana...,None,None,https://en.wikipedia.org/wiki/Content_analysis,2
3,Content analysis is best understood as a broad...,None,None,https://en.wikipedia.org/wiki/Content_analysis,3
4,The simplest and most objective form of conten...,None,None,https://en.wikipedia.org/wiki/Content_analysis,4
...,...,...,...,...,...
85,Much of qualitative coding can be attributed t...,1,assigned labels (sometimes called codes),https://en.wikipedia.org/wiki/Content_analysis,85
86,Coding is considered a process of discovery an...,1,assigned labels (sometimes called codes),https://en.wikipedia.org/wiki/Content_analysis,86
87,"The process can be done manually, which can be...",1,assigned labels (sometimes called codes),https://en.wikipedia.org/wiki/Content_analysis,87
88,After assembling codes it is time to organize ...,1,assigned labels (sometimes called codes),https://en.wikipedia.org/wiki/Content_analysis,88


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [462]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [549]:
for urlTuple in otherPAgeURLS[:3]:
    print(*urlTuple)
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)

https://en.wikipedia.org/wiki/Document 0 documents


KeyboardInterrupt: 

In [471]:
contentParagraphsDF

,paragraph-text,source-paragraph-number,source-paragraph-text,source,paragraph-number
0,Content analysis is the study of documents and...,None,None,https://en.wikipedia.org/wiki/Content_analysis,0
1,Practices and philosophies of content analysis...,None,None,https://en.wikipedia.org/wiki/Content_analysis,1
2,Computers are increasingly used in content ana...,None,None,https://en.wikipedia.org/wiki/Content_analysis,2
3,Content analysis is best understood as a broad...,None,None,https://en.wikipedia.org/wiki/Content_analysis,3
4,The simplest and most objective form of conten...,None,None,https://en.wikipedia.org/wiki/Content_analysis,4
...,...,...,...,...,...
85,Much of qualitative coding can be attributed t...,1,assigned labels (sometimes called codes),https://en.wikipedia.org/wiki/Content_analysis,85
86,Coding is considered a process of discovery an...,1,assigned labels (sometimes called codes),https://en.wikipedia.org/wiki/Content_analysis,86
87,"The process can be done manually, which can be...",1,assigned labels (sometimes called codes),https://en.wikipedia.org/wiki/Content_analysis,87
88,After assembling codes it is time to organize ...,1,assigned labels (sometimes called codes),https://en.wikipedia.org/wiki/Content_analysis,88



# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>



## Section2-1 Website of my interest
Collect the title and url of each campaign case. Then, through the url, I collected the narrative description of the campaign.

In [698]:
# The titles and urls are wraped in <div> <class = "cell grid-item small-6 medium-4 js-fund-tile">
fund_item = gofundSoup.body.findAll('div', {"class":"cell grid-item small-6 medium-4 js-fund-tile"})

items_url = [] 
for num, item in enumerate(fund_item):
    links = item.findAll('a', href=re.compile('/www.gofundme.com/'), class_=False)
    for aTag in links:
        if type(aTag) is bs4.element.Tag:
            relurl = aTag.get('href')
            linkText = re.sub(r'\n','',aTag.text)
            linkText = linkText.strip().split(" "*13)[0]  
            # 13: hard-coding here, try the length of blank between two repeated titles
            items_url.append((relurl, linkText))
items_url # list of (url, title)

[('https://www.gofundme.com/f/justiceforjacobblake',
  'Justice for Jacob Blake'),
 ('https://www.gofundme.com/f/please-help-me-in-the-battle-for-my-life',
  'PLEASE HELP ME IN THE BATTLE FOR MY LIFE'),
 ('https://www.gofundme.com/f/can-you-give-baby-teera-one-life-saving-gene',
  'Can you give Baby Teera one life saving gene?'),
 ('https://www.gofundme.com/f/helping-laura-and-katherine-get-through-my-death',
  'Helping Laura and Katherine get through my death.'),
 ('https://www.gofundme.com/f/help-mattress-mack-save-camp-hope',
  'Help Mattress Mack Save Camp Hope'),
 ('https://www.gofundme.com/f/2succs-the-nolan-family', 'The Nolan Family'),
 ('https://www.gofundme.com/f/brian-page-87-strong', 'Brian Page 87 Strong'),
 ('https://www.gofundme.com/f/save-dogu039s-life', "Save DOGU's Life..."),
 ('https://www.gofundme.com/f/help-feed-victims-of-2020-massive-beirut-explosion',
  'Food for Disaster Victims of Beirut, Lebanon'),
 ('https://www.gofundme.com/f/keith-thomas-jr-recovery-suppor

In [699]:
posts_df['item_num'] = range(len(posts_df['medical_campaign']))
posts_df['item_title'] = ["None"]*len(posts_df['medical_campaign'])
posts_df

,medical_campaign,amount,amount_raised,amount_goal,last_donation,covid_related,item_num,item_title
0,"Kenosha, WI Justice...","[(2.3M, $3.0M ...",2.3M,3.0M,4h,False,0,None
1,"New Milford, NJ PLE...","[(514,718, $600,000 ...","514,718","600,000",2h,False,1,None
2,"Sammamish, WA Can y...","[(464,153, $2.3M ...","464,153",2.3M,3m,False,2,None
3,"Eugene, OR Helping ...","[(423,560, $200,000 ...","423,560","200,000",2d,False,3,None
4,"Houston, TX Help Ma...","[(403,058, $1.0M ...","403,058",1.0M,3mos,False,4,None
5,"Rancho Santa Margarita, CA ...","[(376,271, $500,000 ...","376,271","500,000",4h,False,5,None
6,"Swedesboro, NJ Bria...","[(324,782, $1.5M ...","324,782",1.5M,4h,False,6,None
7,"El Paso, TX Save DO...","[(2.4M, $2.4M ...",2.4M,2.4M,1mo,False,7,None
8,"Pepsi-Cola Sign, NY ...","[(292,796, $250,000 ...","292,796","250,000",1d,True,8,None
9,"Westbury, NY Keith ...","[(289,172, $300,000 ...","289,172","300,000",2h,False,9,None


In [716]:
posts_df['item_title'] = [t[1] for t in items_url ]
posts_df['url'] = [t[0] for t in items_url]
posts_df

,medical_campaign,amount,amount_raised,amount_goal,last_donation,covid_related,item_num,item_title,description,url
0,"Kenosha, WI Justice...","[(2.3M, $3.0M ...",2.3M,3.0M,4h,False,0,Justice for Jacob Blake,[On August 23rd my son was shot multiple times...,https://www.gofundme.com/f/justiceforjacobblake
1,"New Milford, NJ PLE...","[(514,718, $600,000 ...","514,718","600,000",2h,False,1,PLEASE HELP ME IN THE BATTLE FOR MY LIFE,[PLEASE HELP ME IN THE BATTLE FOR MY LIFE. My ...,https://www.gofundme.com/f/please-help-me-in-t...
2,"Sammamish, WA Can y...","[(464,153, $2.3M ...","464,153",2.3M,3m,False,2,Can you give Baby Teera one life saving gene?,[Hello well-wishers. This is our donation page...,https://www.gofundme.com/f/can-you-give-baby-t...
3,"Eugene, OR Helping ...","[(423,560, $200,000 ...","423,560","200,000",2d,False,3,Helping Laura and Katherine get through my death.,"[I am dying of lymphoma. My wife, Laura, has b...",https://www.gofundme.com/f/helping-laura-and-k...
4,"Houston, TX Help Ma...","[(403,058, $1.0M ...","403,058",1.0M,3mos,False,4,Help Mattress Mack Save Camp Hope,[Mattress Mack invites you to help him SAVE CA...,https://www.gofundme.com/f/help-mattress-mack-...
5,"Rancho Santa Margarita, CA ...","[(376,271, $500,000 ...","376,271","500,000",4h,False,5,The Nolan Family,[Our beloved Nolan Family needs our help right...,https://www.gofundme.com/f/2succs-the-nolan-fa...
6,"Swedesboro, NJ Bria...","[(324,782, $1.5M ...","324,782",1.5M,4h,False,6,Brian Page 87 Strong,[ICE HOCKEY STAR CONTINUES HIS FIGHT OFF THE I...,https://www.gofundme.com/f/brian-page-87-strong
7,"El Paso, TX Save DO...","[(2.4M, $2.4M ...",2.4M,2.4M,1mo,False,7,Save DOGU's Life...,[DOĞU’YA GÜNEŞ SİZLERİN DESTEĞİ İLE DOĞSUN … M...,https://www.gofundme.com/f/save-dogu039s-life
8,"Pepsi-Cola Sign, NY ...","[(292,796, $250,000 ...","292,796","250,000",1d,True,8,"Food for Disaster Victims of Beirut, Lebanon",None,https://www.gofundme.com/f/help-feed-victims-o...
9,"Westbury, NY Keith ...","[(289,172, $300,000 ...","289,172","300,000",2h,False,9,Keith Thomas Jr Recovery Support,None,https://www.gofundme.com/f/keith-thomas-jr-rec...


In [718]:
def getTextFromURL(targetURL):
    '''
    To get the text of each campaign through its url collected from main page.
    Input: (str) url
    Output: (list) list of texts
    '''
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    divTag = soup.body.findAll('div',{'class':'o-campaign-story mt3x'})
    l_text = []
    for num, div in enumerate(divTag):
        text = re.sub(r'\n','', div.text)
        l_text.append(text)
        print(text)
    return l_text

# Add None in advance to avoid unmatch of the length of data and dataframe
l_texts = [None]*len(posts_df['medical_campaign']) 
n = 3  # depends on how many cases I want to collect
# Add texts to the dataframe
for num, urlTuple in enumerate(items_url[:n]):
    url = urlTuple[0]
    print(num, url)
    l_texts[num] = getTextFromURL(url)
    posts_df['description'] = l_texts
    

0 https://www.gofundme.com/f/justiceforjacobblake
On August 23rd my son was shot multiple times in the back by a Kenosha Police Department officer, after my son broke up an altercation by an unrelated party. The shooting has now left my son critically injured as he fights for his life. The extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.This fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for Jacob.  A portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.Anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at: Julia Y. Jacksonc/o Ben Crump Law, PLLC122 S. Calhoun StreetTallahassee, FL 32301Attn: Adner Marcelin 
1 ht

In [719]:
posts_df
# the unstandard format, like 2.3M, shoud be transformed later for processing 

,medical_campaign,amount,amount_raised,amount_goal,last_donation,covid_related,item_num,item_title,description,url
0,"Kenosha, WI Justice...","[(2.3M, $3.0M ...",2.3M,3.0M,4h,False,0,Justice for Jacob Blake,[On August 23rd my son was shot multiple times...,https://www.gofundme.com/f/justiceforjacobblake
1,"New Milford, NJ PLE...","[(514,718, $600,000 ...","514,718","600,000",2h,False,1,PLEASE HELP ME IN THE BATTLE FOR MY LIFE,[PLEASE HELP ME IN THE BATTLE FOR MY LIFE. My ...,https://www.gofundme.com/f/please-help-me-in-t...
2,"Sammamish, WA Can y...","[(464,153, $2.3M ...","464,153",2.3M,3m,False,2,Can you give Baby Teera one life saving gene?,[Hello well-wishers. This is our donation page...,https://www.gofundme.com/f/can-you-give-baby-t...
3,"Eugene, OR Helping ...","[(423,560, $200,000 ...","423,560","200,000",2d,False,3,Helping Laura and Katherine get through my death.,None,https://www.gofundme.com/f/helping-laura-and-k...
4,"Houston, TX Help Ma...","[(403,058, $1.0M ...","403,058",1.0M,3mos,False,4,Help Mattress Mack Save Camp Hope,None,https://www.gofundme.com/f/help-mattress-mack-...
5,"Rancho Santa Margarita, CA ...","[(376,271, $500,000 ...","376,271","500,000",4h,False,5,The Nolan Family,None,https://www.gofundme.com/f/2succs-the-nolan-fa...
6,"Swedesboro, NJ Bria...","[(324,782, $1.5M ...","324,782",1.5M,4h,False,6,Brian Page 87 Strong,None,https://www.gofundme.com/f/brian-page-87-strong
7,"El Paso, TX Save DO...","[(2.4M, $2.4M ...",2.4M,2.4M,1mo,False,7,Save DOGU's Life...,None,https://www.gofundme.com/f/save-dogu039s-life
8,"Pepsi-Cola Sign, NY ...","[(292,796, $250,000 ...","292,796","250,000",1d,True,8,"Food for Disaster Victims of Beirut, Lebanon",None,https://www.gofundme.com/f/help-feed-victims-o...
9,"Westbury, NY Keith ...","[(289,172, $300,000 ...","289,172","300,000",2h,False,9,Keith Thomas Jr Recovery Support,None,https://www.gofundme.com/f/keith-thomas-jr-rec...


In [711]:
# An example of one case page
targetURL = "https://www.gofundme.com/f/justiceforjacobblake"
r = requests.get(targetURL)

soup = bs4.BeautifulSoup(r.text, 'html.parser')
a = soup.body.findAll('div',{'class':'o-campaign-story mt3x'})
for num, divTag in enumerate(a):
    print(divTag.text)

On August 23rd my son was shot multiple times in the back by a Kenosha Police Department officer, after my son broke up an altercation by an unrelated party. The shooting has now left my son critically injured as he fights for his life. The extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

This fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for Jacob.  A portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

Anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
Julia Y. Jackson
c/o Ben Crump Law, PLLC
122 S. Calhoun Street
Tallahassee, FL 32301
Attn: Adner Marcelin
 


## Section 2-2 Complement  
I also used another webpage to do Section 2, because the website of my interest, GoFundMe, has different structure at the two page levels that I am interested in.  
So I choose another website, just in order to learn and practice scraping context in different levels through urls.

In [8]:
import requests
import bs4
mainpage = "https://www.thehastingscenter.org/our-focus-areas/focus-area-science-technology/"
mainpageRequest = requests.get(mainpage)
mainpageSoup = bs4.BeautifulSoup(mainpageRequest.text, 'html.parser')
print(mainpageSoup)

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={licenseKey:"13cdda6223",applicationID:"64691492"};window.NREUM||(NREUM={}),__nr_require=function(e,t,n){function r(n){if(!t[n]){var i=t[n]={exports:{}};e[n][0].call(i.exports,function(t){var i=e[n][1][t];return r(i||t)},i,i.exports)}return t[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(e,t,n){function r(){}function i(e,t,n){return function(){return o(e,[u.now()].concat(c(arguments)),t?null:this,n),t?void 0:this}}var o=e("handle"),a=e(6),c=e(7),f=e("ee").get("tracer"),u=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var d=["setPageViewName","setCustomAttribute","setErrorHandler","finished","addToTrace","inlineHit","addRelease"],p="api-",l=p+"ixn-";a(d,function(e,t){s[t]=i(p+t,!0,"api")}),s.addPageAction=i(p+"addPageAction",!0),s.setCurrentRouteName=

In [21]:
mainpagePTags = mainpageSoup.body.findAll('p')

mainpageParagraphs = []
for pTag in mainpagePTags:
    if pTag.text !='':
        mainpageParagraphs.append(pTag.text)
mainpageParagraphs

# convert to a DataFrame
mainpageDF = pandas.DataFrame({'paragraph-text':mainpageParagraphs})
print(mainpageDF)

                                       paragraph-text
0   Advances in genomics, neuroscience, synthetic ...
1   The Art of Flourishing: Conversations on Disab...
2   Actionable Ethics Oversight of Human Animal Ch...
3   HIDE: Homeland Security, Biometric Identificat...
4   Control and Responsible Innovation in the Deve...
5          Psychiatric Times: The Ethics of Reopening
6                        The Genetics of Intelligence
7                             Artificial Intelligence
8                   Clinical Trials & Research Ethics
9                   Conflicts of Interest in Research
10                                       Neuroscience
11                                        Enhancement
12                                           Genetics
13                                  Synthetic Biology
14                               Privacy & Biobanking


In [22]:
otherPageUrls = []
for num, pTag in enumerate(mainpagePTags):
    tagLinks = pTag.findAll('a', href = re.compile('https://www.thehastingscenter.org/'), class_=False)
    for aTag in tagLinks:
        url = aTag.get('href')
        linkText = aTag.text
        otherPageUrls.append((num, url, linkText)) 
print(otherPageUrls)

[(13, 'https://www.thehastingscenter.org/who-we-are/our-research/current-projects/the-art-of-flourishing-conversations-on-disability-and-technology/', 'The Art of Flourishing: Conversations on Disability'), (14, 'https://www.thehastingscenter.org/who-we-are/our-research/current-projects/actionable-ethics-oversight-human-animal-chimera-research/', 'Actionable Ethics Oversight of Human Animal Chimera Research'), (15, 'https://www.thehastingscenter.org/who-we-are/our-research/selected-past-projects/hide-homeland-security-biometric-identification-and-personal-detection-ethics/', 'HIDE: Homeland Security, Biometric Identification and Personal Detection Ethics'), (16, 'https://www.thehastingscenter.org/who-we-are/our-research/current-projects/control-and-responsible-innovation-in-the-development-of-autonomous-machines/', 'Control and Responsible Innovation in the Development of Autonomous Machines'), (18, 'https://www.thehastingscenter.org/publications-resources/special-reports-2/the-genetic

In [37]:
mainpageDF['source'] = [mainpage] * len(mainpageDF['paragraph-text'])
mainpageDF['paragraph-number'] = range(len(mainpageDF['paragraph-text']))
mainpageDF

,paragraph-text,source,paragraph-number,source-paragraph-number,source-paragraph-text
0,"Advances in genomics, neuroscience, synthetic ...",https://www.thehastingscenter.org/our-focus-ar...,0,None,None
1,The Art of Flourishing: Conversations on Disab...,https://www.thehastingscenter.org/our-focus-ar...,1,None,None
2,Actionable Ethics Oversight of Human Animal Ch...,https://www.thehastingscenter.org/our-focus-ar...,2,None,None
3,"HIDE: Homeland Security, Biometric Identificat...",https://www.thehastingscenter.org/our-focus-ar...,3,None,None
4,Control and Responsible Innovation in the Deve...,https://www.thehastingscenter.org/our-focus-ar...,4,None,None
...,...,...,...,...,...
316,,https://www.thehastingscenter.org/our-focus-ar...,316,23,Enhancement
317,,https://www.thehastingscenter.org/our-focus-ar...,317,23,Enhancement
318,,https://www.thehastingscenter.org/our-focus-ar...,318,23,Enhancement
319,,https://www.thehastingscenter.org/our-focus-ar...,319,23,Enhancement


In [34]:
mainpageDF['source-paragraph-number'] = [None] * len(mainpageDF['paragraph-text'])
mainpageDF['source-paragraph-text'] = [None] * len(mainpageDF['paragraph-text'])

def getTextFromMainPage(sourceParNum, targetURL, sourceText):
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\n', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)
   
for urlTuple in otherPageUrls[:10]:
    print(*urlTuple)
    mainpageDF = mainpageDF.append(getTextFromMainPage(*urlTuple),ignore_index=True)

13 https://www.thehastingscenter.org/who-we-are/our-research/current-projects/the-art-of-flourishing-conversations-on-disability-and-technology/ The Art of Flourishing: Conversations on Disability
14 https://www.thehastingscenter.org/who-we-are/our-research/current-projects/actionable-ethics-oversight-human-animal-chimera-research/ Actionable Ethics Oversight of Human Animal Chimera Research
15 https://www.thehastingscenter.org/who-we-are/our-research/selected-past-projects/hide-homeland-security-biometric-identification-and-personal-detection-ethics/ HIDE: Homeland Security, Biometric Identification and Personal Detection Ethics
16 https://www.thehastingscenter.org/who-we-are/our-research/current-projects/control-and-responsible-innovation-in-the-development-of-autonomous-machines/ Control and Responsible Innovation in the Development of Autonomous Machines
18 https://www.thehastingscenter.org/publications-resources/special-reports-2/the-genetics-of-intelligence/ The Genetics of Intel

In [36]:
mainpageDF

,paragraph-text,source,paragraph-number,source-paragraph-number,source-paragraph-text
0,"Advances in genomics, neuroscience, synthetic ...",https://www.thehastingscenter.org/our-focus-ar...,0,None,None
1,The Art of Flourishing: Conversations on Disab...,https://www.thehastingscenter.org/our-focus-ar...,1,None,None
2,Actionable Ethics Oversight of Human Animal Ch...,https://www.thehastingscenter.org/our-focus-ar...,2,None,None
3,"HIDE: Homeland Security, Biometric Identificat...",https://www.thehastingscenter.org/our-focus-ar...,3,None,None
4,Control and Responsible Innovation in the Deve...,https://www.thehastingscenter.org/our-focus-ar...,4,None,None
...,...,...,...,...,...
316,,https://www.thehastingscenter.org/category/enh...,10,23,Enhancement
317,,https://www.thehastingscenter.org/category/enh...,11,23,Enhancement
318,,https://www.thehastingscenter.org/category/enh...,12,23,Enhancement
319,,https://www.thehastingscenter.org/category/enh...,13,23,Enhancement


## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [ ]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [ ]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [ ]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [ ]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [559]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [ ]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. Lets start by looking at the Shakespeare files in the `data` directory 

In [ ]:
with open('./data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

By the way, depending on your working directory, you might get errors such as: [Errno 2] No such file or directory: '../data/Shakespeare/midsummer_nights_dream.txt.' Don't panic, it's nothing, just check your working directory. 

Then to load all the files in `./data/Shakespeare` we can use a for loop with `scandir`:

In [ ]:
targetDir = './data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path, encoding="utf-8") as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [ ]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [564]:
#information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

ConnectionError: HTTPSConnectionPool(host='raw.githubusercontent.com', port=443): Max retries exceeded with url: /Computational-Content-Analysis-2018/Data-Files/master/1-intro/Content%20Analysis%2018.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f89d00cd160>: Failed to establish a new connection: [Errno 61] Connection refused'))

It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [565]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [ ]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [ ]:
print(readPDF(infoExtractionBytes)[:550])

From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [ ]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud_2020` so we can use it later without having to retype it.

In [ ]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [ ]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [ ]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


Section3  
Extract a paper -- Sources of Segregation in Social Networks: A Novel Approach Using Facebook
Get all the sentences that contain the keyword

In [587]:
# From url: Sources of Segregation in Social Networks: A Novel Approach Using Facebook
paper_pdf = "http://sage.cnpereading.com/paragraph/download/?doi=10.1177/0003122417705656"
inforequest = requests.get(paper_pdf, stream=True)
text = readPDF(io.BytesIO(inforequest.content))

In [589]:
# These are the keywords of the paper
keywords=['segregation', 'ethnicity', 'gender', 'social networks', 'Facebook']

In [683]:
t=text
t

'705656 ASRXXX10.1177/0003122417705656American Sociological ReviewHofstra et al.\n\n2017\n\nSources of Segregation in  \nSocial Networks: A Novel \nApproach Using Facebook\n\nAmerican Sociological Review\n2017, Vol. 82(3) 625 –656\n© American Sociological  \nAssociation 2017\nDOI: 10.1177/0003122417705656\nhttps://doi.org/10.1177/0003122417705656\njournals.sagepub.com/home/asr\n\nBas Hofstra,a Rense Corten,a  \nFrank van Tubergen,b and Nicole B. Ellisonc\n\nAbstract\nMost research on segregation in social networks considers small circles of strong ties, and \nlittle is known about segregation among the much larger number of weaker ties. This article \nproposes a novel approach to the study of these more extended networks, through the use \nof  data  on  personal  ties  in  an  online  social  network.  We  illustrate  this  method’s  potential \nby  describing  and  explaining  the  degree  of  ethnic  and  gender  segregation  on  Facebook \namong  a  representative  survey  of  adole

In [720]:
# I extract all the sentences that include the keywords
WordSentence = {}
for word in keywords:
    WordSentence[word] = []

l_keywords = []
l_sentences = []
for line in re.findall(r'[^.]{20,}', t):
    line = re.sub(r'\n','',line)
    if re.search(r'[p]\s*[p]', line) == None and re.search(r':[\d]', line) == None:
        for word in keywords:
            if word in line:
                sentence = re.sub(r'\n\d*','',line.strip())
                l_keywords.append(word)
                l_sentences.append(sentence)
        
len(l_keywords) == len(l_sentences)

True

In [721]:
# dataframe of keyword and its corresponding sentences
# Notice that the same sentence could be relevant to different keywords.
content_df = pandas.DataFrame({'keyword' : l_keywords, 'sentence' : l_sentences})
content_df

,keyword,sentence
0,segregation,EllisoncAbstractMost research on segregation i...
1,social networks,EllisoncAbstractMost research on segregation i...
2,segregation,We illustrate this method’s potential by ...
3,gender,We illustrate this method’s potential by ...
4,Facebook,We illustrate this method’s potential by ...
...,...,...
197,Facebook,"“Tastes, Ties, and Time: A New Social Network ..."
198,Facebook,“Beyond and Below Racial Homophily: ERG Models...
199,segregation,"Specifically, he studies (early) adoption of ..."
200,social networks,"His research topics are social networks, relig..."


### Other sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg 

